In [ ]:
import os, cv2, random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.models import load_model

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../data"]).decode("utf8"))

In [ ]:
TRAIN_1_DIR = '../data/train/1/'
TRAIN_0_DIR = '../data/train/0/'


ROWS = 36
COLS = 36
CHANNELS = 1

train_1 =   [TRAIN_1_DIR+i for i in os.listdir(TRAIN_1_DIR)]
train_0 =   [TRAIN_0_DIR+i for i in os.listdir(TRAIN_0_DIR)]

train_images = train_1[:5000] + train_0[:5000]

print(len(train_images))
print(len(train_1))
print(len(train_0))

In [ ]:
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) #
    return cv2.resize(img, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)


def prep_data(images):
    count = len(images)
    data = np.ndarray((count, ROWS, COLS), dtype=np.uint8)

    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        if i%250 == 0: print('Processed {} of {}'.format(i, count))
    
    return data


train = prep_data(train_images)
train_and_label = zip(train, ([1]*len(train_1)) + ([0]*len(train_0)))

print("Train shape: {}".format(train.shape))

In [ ]:
def show_example(idx):
    face = read_image(train_1[idx])
    noface = read_image(train_0[idx])
    pair = np.concatenate((face, noface), axis=1)
    plt.figure(figsize=(10,5))
    plt.imshow(pair, cmap='gray')
    plt.show()
    

show_example(1)

In [ ]:
show_example(2)

In [ ]:
show_example(4)

In [ ]:
train_and_label = list(zip(train, ([1]*5000 + ([0]*5000))))

In [ ]:
train_and_label[5000]

In [ ]:
random.shuffle(train_and_label)

In [ ]:
list(map(lambda x: x[1], train_and_label))[:20]

In [ ]:
images = np.array(list(map(lambda x: x[0], train_and_label)))
images.resize((10000, 36, 36, 1))

labels = np.array(list(map(lambda x: x[1], train_and_label)))

In [ ]:
optimizer = RMSprop(lr=1e-4)
objective = 'binary_crossentropy'


def faceRecognition():
    
    model = Sequential()

    model.add(Conv2D(4, 5, strides=(1,1), border_mode='same',
                     input_shape=(36, 36, 1), data_format="channels_last", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, 3, strides=(1,1), border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    return model


model = faceRecognition()

In [ ]:
nb_epoch = 10
batch_size = 32

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('accuracy'))
        self.val_losses.append(logs.get('val_loss'))
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')        
        
def train_model():
    
    history = LossHistory()
    model.fit(images, labels, batch_size=batch_size, nb_epoch=nb_epoch,
              validation_split=0.25, verbose=0, shuffle=True, callbacks=[history, early_stopping])
    return history

history = train_model()

In [ ]:
loss = history.losses
val_loss = history.val_losses

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('VGG-16 Loss Trend')
plt.plot(loss, 'blue', label='Training Acc')
plt.plot(val_loss, 'green', label='Validation Acc')
plt.xticks(range(0,nb_epoch)[0::2])
plt.legend()
plt.show()

In [ ]:
test_images = train_1[5500:6500] + train_0[5500:6500]
test = prep_data(test_images)
tests = test
tests.resize((2000, 36, 36, 1))
predictions = model.predict(tests, verbose=0)

In [ ]:
len(list(filter(lambda x: x[0] == x[1], list(zip(([0]*1000 + [1]*1000), list(map(lambda x: int(x+00.1), predictions)))))))

In [ ]:
model.save('my_model.h5')
del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')
predictions = model.predict(tests, verbose=0)
len(list(filter(lambda x: x[0] == x[1], list(zip(([0]*1000 + [1]*1000), list(map(lambda x: int(x+00.1), predictions)))))))